# Exercise - RNN Classification

In this notebook, we will perform a classification task using RNNs (i.e., a sequence to value prediction). We have hourly power consumption of households for 12 hours. Based on this, we will determine whether the power grid is strained (1) or not (0). 

Therefore, use the columns from `Hour 0` to `Hour 11` to predict the `target` column in the `power.csv` data set.

Hint1: Use Tutorial 1 for help.

Hint2: Don't forget to adjust the number of neurons in the input layers correctly. Otherwise, you will run into errors.

In [1]:
import tensorflow as tf
from tensorflow import keras
from sklearn.metrics import mean_squared_error


# Common imports
import numpy as np
import os
import pandas as pd

# to make this notebook's output stable across runs
np.random.seed(42)

# To plot pretty figures
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
mpl.rc('axes', labelsize=14)
mpl.rc('xtick', labelsize=12)
mpl.rc('ytick', labelsize=12)



# Read the Dataset

In [2]:
power = pd.read_csv('power.csv')

power.head()

,Hour 0,Hour 1,Hour 2,Hour 3,Hour 4,Hour 5,Hour 6,Hour 7,Hour 8,Hour 9,Hour 10,Hour 11,target
0,2.550633,2.523400,2.582333,2.541667,2.475733,2.476233,2.455800,2.447200,2.441733,3.146133,2.661733,2.576000,1
1,1.596933,1.619567,2.473733,2.731133,2.431133,2.479667,1.690200,1.332133,1.375167,1.050900,0.585900,2.651900,1
2,0.534933,0.540467,0.575367,0.526500,0.521900,0.565333,1.426467,0.602067,0.547433,0.525067,1.270300,0.393767,0
3,1.085867,0.651233,0.634600,0.653000,0.646067,0.628400,0.611067,0.612533,0.660100,0.606067,1.471867,0.834533,0
4,0.456000,0.286300,0.310833,0.250933,0.277667,0.308633,0.610400,1.563533,1.421867,3.324400,3.207567,1.425433,1


In [3]:
power.shape

(1417, 13)

# Split the Data



In [4]:
# First 1000 days are for train
train = power.iloc[:1000]

# Remaining 417 days are for test
test = power.iloc[-417:]

In [5]:
train.shape

(1000, 13)

In [6]:
test.shape

(417, 13)

# Create Input and Target values

The first 12 columns (hourly data) will be input to predict the last column (i.e., target)

In [7]:
# The first 12 columns (from 0 to 11) are inputs

train_inputs = train.iloc[:,:12]

## Add one more dimension to make it ready for RNNs

In [8]:
#Create an additional dimension for train

train_x = np.array(train_inputs).reshape(1000,12,1)

train_x.shape 

(1000, 12, 1)

## Set the target

In [9]:
# The last column is TARGET

train_target = train.iloc[:,-1]

## Repeat for TEST

In [10]:
test.shape

(417, 13)

In [11]:
# The first 12 columns are inputs

test_inputs = test.iloc[:,:12]

In [12]:
#Create an additional dimension for test

test_x = np.array(test_inputs).reshape(417,12,1)

test_x.shape 

(417, 12, 1)

In [13]:
# The last column is TARGET

test_target = test.iloc[:,-1]

In [14]:
from sklearn.dummy import DummyClassifier

dummy_clf = DummyClassifier(strategy="most_frequent")

dummy_clf.fit(train_inputs, train_target)

DummyClassifier(strategy='most_frequent')

In [15]:
#Baseline Train Accuracy
from sklearn.metrics import accuracy_score
dummy_train_pred = dummy_clf.predict(train_inputs)

baseline_train_acc = accuracy_score(train_target, dummy_train_pred)

print('Baseline Train Accuracy: {}' .format(baseline_train_acc))

Baseline Train Accuracy: 0.505


In [16]:
#Baseline Test Accuracy
dummy_test_pred = dummy_clf.predict(test_inputs)

baseline_test_acc = accuracy_score(test_target, dummy_test_pred)

print('Baseline Test Accuracy: {}' .format(baseline_test_acc))

Baseline Test Accuracy: 0.49640287769784175


# Build a normal (cross-sectional) NN

This model assumes that the data is NOT a time-series data set. It treats the data as cross-sectional and the columns being independent of each other.

In [17]:
model = keras.models.Sequential([

    keras.layers.Flatten(input_shape=[12, 1]),
    keras.layers.Dense(12, activation='relu'),
    keras.layers.Dense(1, activation='sigmoid')  
    
])

In [18]:
np.random.seed(42)
tf.random.set_seed(42)

optimizer = tf.keras.optimizers.Nadam(learning_rate=0.01)

model.compile(loss="binary_crossentropy", optimizer=optimizer, metrics=['accuracy'])

history = model.fit(train_inputs, train_target, epochs=50,
                    validation_data=(test_inputs, test_target))

Epoch 1/50
32/32 [==============================] - 1s 7ms/step - loss: 0.6846 - accuracy: 0.5580 - val_loss: 0.6636 - val_accuracy: 0.5851
Epoch 2/50
32/32 [==============================] - 0s 2ms/step - loss: 0.6039 - accuracy: 0.6930 - val_loss: 0.5964 - val_accuracy: 0.6906
Epoch 3/50
32/32 [==============================] - 0s 3ms/step - loss: 0.5540 - accuracy: 0.7220 - val_loss: 0.6347 - val_accuracy: 0.6211
Epoch 4/50
32/32 [==============================] - 0s 2ms/step - loss: 0.5186 - accuracy: 0.7540 - val_loss: 0.5430 - val_accuracy: 0.7050
Epoch 5/50
32/32 [==============================] - 0s 2ms/step - loss: 0.4962 - accuracy: 0.7480 - val_loss: 0.5332 - val_accuracy: 0.7050
Epoch 6/50
32/32 [==============================] - 0s 2ms/step - loss: 0.4782 - accuracy: 0.7520 - val_loss: 0.5202 - val_accuracy: 0.7026
Epoch 7/50
32/32 [==============================] - 0s 2ms/step - loss: 0.4725 - accuracy: 0.7570 - val_loss: 0.5278 - val_accuracy: 0.7362
Epoch 8/50
32/32 [==

In [19]:
# evaluate the model

scores = model.evaluate(test_inputs, test_target, verbose=0)
scores

print("%s: %.2f" % (model.metrics_names[0], scores[0]))
print("%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))

#72.18% accuracy is much better than the baseline that is just around 50%


loss: 0.51
accuracy: 73.62%


# Build a simple RNN with one layer

In [20]:
n_steps = 12
n_inputs = 1


model = keras.models.Sequential([
    
    keras.layers.SimpleRNN(12, input_shape=[n_steps, n_inputs]),  
    keras.layers.Dense(1, activation='sigmoid')
])

In [21]:
from tensorflow.keras.callbacks import EarlyStopping
earlystop = EarlyStopping(monitor='val_loss', patience=5, verbose=1, mode='auto')
callback = [earlystop]

In [22]:
np.random.seed(42)
tf.random.set_seed(42)

optimizer = tf.keras.optimizers.Nadam(learning_rate=0.01)

# If multiclass, use "sparse_categorical_crossentropy" as the loss function
model.compile(loss="binary_crossentropy", optimizer=optimizer, metrics=['accuracy'])


history = model.fit(train_inputs, train_target, epochs=50,
                    validation_data=(test_inputs, test_target), callbacks=callback)

Epoch 1/50
32/32 [==============================] - 1s 10ms/step - loss: 0.5539 - accuracy: 0.7210 - val_loss: 0.5144 - val_accuracy: 0.7338
Epoch 2/50
32/32 [==============================] - 0s 4ms/step - loss: 0.4957 - accuracy: 0.7420 - val_loss: 0.5413 - val_accuracy: 0.7098
Epoch 3/50
32/32 [==============================] - 0s 4ms/step - loss: 0.4831 - accuracy: 0.7530 - val_loss: 0.8122 - val_accuracy: 0.5707
Epoch 4/50
32/32 [==============================] - 0s 4ms/step - loss: 0.5057 - accuracy: 0.7370 - val_loss: 0.5483 - val_accuracy: 0.7074
Epoch 5/50
32/32 [==============================] - 0s 4ms/step - loss: 0.4727 - accuracy: 0.7630 - val_loss: 0.5359 - val_accuracy: 0.7074
Epoch 6/50
32/32 [==============================] - 0s 4ms/step - loss: 0.4651 - accuracy: 0.7680 - val_loss: 0.5105 - val_accuracy: 0.7122
Epoch 7/50
32/32 [==============================] - 0s 4ms/step - loss: 0.4665 - accuracy: 0.7600 - val_loss: 0.5290 - val_accuracy: 0.7338
Epoch 8/50
32/32 [=

In [23]:
# evaluate the model

scores = model.evaluate(test_inputs, test_target, verbose=0)
scores

print("%s: %.2f" % (model.metrics_names[0], scores[0]))
print("%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))

# This is the best model so far.

loss: 0.51
accuracy: 72.66%


# Build a simple RNN with two or more layers

In [24]:
n_steps = 12
n_inputs = 1


model = keras.models.Sequential([
    keras.layers.SimpleRNN(12, return_sequences=True, input_shape=[n_steps, n_inputs] ),
    keras.layers.SimpleRNN(22, return_sequences=True),
    keras.layers.SimpleRNN(32), 
    keras.layers.Dense(1, activation='sigmoid')  
])


In [25]:
np.random.seed(42)
tf.random.set_seed(42)

optimizer = keras.optimizers.Nadam(learning_rate=0.01)

model.compile(loss="binary_crossentropy", optimizer=optimizer, metrics=['accuracy'])

history = model.fit(train_inputs, train_target, epochs=20,
                   validation_data = (test_inputs, test_target), callbacks=callback)

Epoch 1/20
32/32 [==============================] - 3s 19ms/step - loss: 0.6534 - accuracy: 0.6260 - val_loss: 0.6593 - val_accuracy: 0.6691
Epoch 2/20
32/32 [==============================] - 0s 7ms/step - loss: 0.5499 - accuracy: 0.7080 - val_loss: 0.6406 - val_accuracy: 0.6715
Epoch 3/20
32/32 [==============================] - 0s 7ms/step - loss: 0.5162 - accuracy: 0.7330 - val_loss: 1.1616 - val_accuracy: 0.5228
Epoch 4/20
32/32 [==============================] - 0s 7ms/step - loss: 0.5483 - accuracy: 0.7220 - val_loss: 0.5380 - val_accuracy: 0.7146
Epoch 5/20
32/32 [==============================] - 0s 7ms/step - loss: 0.4827 - accuracy: 0.7670 - val_loss: 0.5690 - val_accuracy: 0.7026
Epoch 6/20
32/32 [==============================] - 0s 7ms/step - loss: 0.4682 - accuracy: 0.7580 - val_loss: 0.5239 - val_accuracy: 0.7122
Epoch 7/20
32/32 [==============================] - 0s 7ms/step - loss: 0.4738 - accuracy: 0.7670 - val_loss: 0.5350 - val_accuracy: 0.7506
Epoch 8/20
32/32 [=

In [26]:
# evaluate the model

scores = model.evaluate(test_inputs, test_target, verbose=0)
scores

print("%s: %.2f" % (model.metrics_names[0], scores[0]))
print("%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))

# This is not as good as the simple RNN

loss: 0.58
accuracy: 71.70%


# Build a LSTM with one layer

In [27]:
n_steps = 12
n_inputs = 1

model = keras.models.Sequential([
    
    keras.layers.LSTM(12, input_shape=[n_steps, n_inputs]),
    keras.layers.Dense(1, activation='sigmoid')
])

In [28]:
np.random.seed(42)
tf.random.set_seed(42)

optimizer = keras.optimizers.Nadam(learning_rate=0.01)

model.compile(loss="binary_crossentropy", optimizer=optimizer, metrics=['accuracy'])

history = model.fit(train_inputs, train_target, epochs=20,
                   validation_data = (test_inputs, test_target), callbacks=callback)

Epoch 1/20
32/32 [==============================] - 2s 18ms/step - loss: 0.5956 - accuracy: 0.6750 - val_loss: 0.5621 - val_accuracy: 0.7242
Epoch 2/20
32/32 [==============================] - 0s 6ms/step - loss: 0.5056 - accuracy: 0.7400 - val_loss: 0.6142 - val_accuracy: 0.6835
Epoch 3/20
32/32 [==============================] - 0s 5ms/step - loss: 0.4898 - accuracy: 0.7500 - val_loss: 0.7647 - val_accuracy: 0.5875
Epoch 4/20
32/32 [==============================] - 0s 5ms/step - loss: 0.4776 - accuracy: 0.7560 - val_loss: 0.5371 - val_accuracy: 0.7170
Epoch 5/20
32/32 [==============================] - 0s 5ms/step - loss: 0.4717 - accuracy: 0.7610 - val_loss: 0.5374 - val_accuracy: 0.7098
Epoch 6/20
32/32 [==============================] - 0s 5ms/step - loss: 0.4646 - accuracy: 0.7610 - val_loss: 0.5187 - val_accuracy: 0.7050
Epoch 7/20
32/32 [==============================] - 0s 5ms/step - loss: 0.4664 - accuracy: 0.7610 - val_loss: 0.5253 - val_accuracy: 0.7554
Epoch 8/20
32/32 [=

In [29]:
# evaluate the model

scores = model.evaluate(test_inputs, test_target, verbose=0)
scores

print("%s: %.2f" % (model.metrics_names[0], scores[0]))
print("%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))

# Best model so far.

loss: 0.52
accuracy: 72.18%


# Build a LSTM with two or more layers

In [30]:
n_steps = 12
n_inputs = 1

model = keras.models.Sequential([
    keras.layers.LSTM(12, return_sequences=True, input_shape=[n_steps, n_inputs]),
    keras.layers.LSTM(12, return_sequences=True),
    keras.layers.LSTM(12),
    keras.layers.Dense(1, activation='sigmoid')
])

In [31]:
np.random.seed(42)
tf.random.set_seed(42)

optimizer = keras.optimizers.Nadam(learning_rate=0.01)

model.compile(loss="binary_crossentropy", optimizer=optimizer, metrics=['accuracy'])

history = model.fit(train_inputs, train_target, epochs=20,
                   validation_data = (test_inputs, test_target), callbacks=callback)

Epoch 1/20
32/32 [==============================] - 5s 42ms/step - loss: 0.6229 - accuracy: 0.6460 - val_loss: 0.6533 - val_accuracy: 0.6331
Epoch 2/20
32/32 [==============================] - 0s 13ms/step - loss: 0.5545 - accuracy: 0.7190 - val_loss: 0.6651 - val_accuracy: 0.6475
Epoch 3/20
32/32 [==============================] - 0s 13ms/step - loss: 0.5315 - accuracy: 0.7340 - val_loss: 0.9075 - val_accuracy: 0.5324
Epoch 4/20
32/32 [==============================] - 0s 13ms/step - loss: 0.5259 - accuracy: 0.7360 - val_loss: 0.5386 - val_accuracy: 0.7098
Epoch 5/20
32/32 [==============================] - 0s 13ms/step - loss: 0.4911 - accuracy: 0.7590 - val_loss: 0.5611 - val_accuracy: 0.7026
Epoch 6/20
32/32 [==============================] - 0s 13ms/step - loss: 0.4730 - accuracy: 0.7590 - val_loss: 0.5141 - val_accuracy: 0.7098
Epoch 7/20
32/32 [==============================] - 0s 13ms/step - loss: 0.4714 - accuracy: 0.7500 - val_loss: 0.5426 - val_accuracy: 0.7530
Epoch 8/20
32

In [32]:
# evaluate the model

scores = model.evaluate(test_inputs, test_target, verbose=0)
scores

print("%s: %.2f" % (model.metrics_names[0], scores[0]))
print("%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))

# Worse NN model

loss: 0.51
accuracy: 77.70%


# Build a GRU with one layer

In [33]:
n_steps = 12
n_inputs = 1

model = keras.models.Sequential([
    keras.layers.GRU(20, input_shape=[n_steps, n_inputs]),
    keras.layers.Dense(1, activation='sigmoid')
])

In [34]:
np.random.seed(42)
tf.random.set_seed(42)

optimizer = keras.optimizers.Nadam(learning_rate=0.01)

model.compile(loss="binary_crossentropy", optimizer=optimizer, metrics=['accuracy'])

history = model.fit(train_inputs, train_target, epochs=20,
                   validation_data = (test_inputs, test_target), callbacks=callback)

Epoch 1/20
32/32 [==============================] - 2s 18ms/step - loss: 0.6199 - accuracy: 0.6340 - val_loss: 0.5543 - val_accuracy: 0.7314
Epoch 2/20
32/32 [==============================] - 0s 6ms/step - loss: 0.5118 - accuracy: 0.7300 - val_loss: 0.5957 - val_accuracy: 0.7002
Epoch 3/20
32/32 [==============================] - 0s 6ms/step - loss: 0.4804 - accuracy: 0.7450 - val_loss: 0.9393 - val_accuracy: 0.5612
Epoch 4/20
32/32 [==============================] - 0s 6ms/step - loss: 0.4763 - accuracy: 0.7440 - val_loss: 0.5414 - val_accuracy: 0.7050
Epoch 5/20
32/32 [==============================] - 0s 6ms/step - loss: 0.4662 - accuracy: 0.7530 - val_loss: 0.5464 - val_accuracy: 0.7098
Epoch 6/20
32/32 [==============================] - 0s 6ms/step - loss: 0.4576 - accuracy: 0.7560 - val_loss: 0.5163 - val_accuracy: 0.7098
Epoch 7/20
32/32 [==============================] - 0s 5ms/step - loss: 0.4623 - accuracy: 0.7490 - val_loss: 0.5207 - val_accuracy: 0.7626
Epoch 8/20
32/32 [=

In [35]:
# evaluate the model

scores = model.evaluate(test_inputs, test_target, verbose=0)
scores

print("%s: %.2f" % (model.metrics_names[0], scores[0]))
print("%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))

loss: 0.52
accuracy: 72.90%


# Build a GRU with two or more layers

In [36]:
n_steps = 12
n_inputs = 1

model = keras.models.Sequential([
    keras.layers.GRU(10, return_sequences=True, input_shape=[n_steps, n_inputs]),
    keras.layers.GRU(10, return_sequences=True),
    keras.layers.GRU(10),
    keras.layers.Dense(1, activation='sigmoid')
])

In [37]:
np.random.seed(42)
tf.random.set_seed(42)

optimizer = keras.optimizers.Nadam(learning_rate=0.01)

model.compile(loss="binary_crossentropy", optimizer=optimizer, metrics=['accuracy'])

history = model.fit(train_inputs, train_target, epochs=20,
                   validation_data = (test_inputs, test_target), callbacks=callback)

Epoch 1/20
32/32 [==============================] - 6s 42ms/step - loss: 0.5866 - accuracy: 0.6860 - val_loss: 0.5607 - val_accuracy: 0.7314
Epoch 2/20
32/32 [==============================] - 0s 14ms/step - loss: 0.5140 - accuracy: 0.7350 - val_loss: 0.6104 - val_accuracy: 0.6954
Epoch 3/20
32/32 [==============================] - 0s 13ms/step - loss: 0.4944 - accuracy: 0.7430 - val_loss: 0.9437 - val_accuracy: 0.5444
Epoch 4/20
32/32 [==============================] - 0s 13ms/step - loss: 0.5122 - accuracy: 0.7330 - val_loss: 0.5340 - val_accuracy: 0.7122
Epoch 5/20
32/32 [==============================] - 0s 13ms/step - loss: 0.4771 - accuracy: 0.7530 - val_loss: 0.5609 - val_accuracy: 0.7122
Epoch 6/20
32/32 [==============================] - 0s 12ms/step - loss: 0.4680 - accuracy: 0.7620 - val_loss: 0.5216 - val_accuracy: 0.7242
Epoch 7/20
32/32 [==============================] - 0s 12ms/step - loss: 0.4686 - accuracy: 0.7510 - val_loss: 0.5442 - val_accuracy: 0.7554
Epoch 8/20
32

In [38]:
# evaluate the model

scores = model.evaluate(test_inputs, test_target, verbose=0)
scores

print("%s: %.2f" % (model.metrics_names[0], scores[0]))
print("%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))

# This final model is the best, edging LSTM with one layer

loss: 0.51
accuracy: 76.02%
